In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import os
import sys

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
!mkdir pdfs

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

225

In [ ]:
os.environ['GOOGLE_API_KEY'] = 'XXXX'

In [ ]:
embeddings=GooglePalmEmbeddings()

In [ ]:
query_result = embeddings.embed_query("Hello World")

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'XXXX')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchainpinecone" # put in the name of your pinecone index here

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
query = "what is the new tax regime"

In [ ]:
docs = docsearch.similarity_search(query, k=3)

In [ ]:
docs

[Document(page_content='9,00,000  -49,400  -18,200  -7,800  46,800  \n10,00,000  -54,600  -23,400  -13,000  28,600  \n11,00,000  -70,200  -28,600  -18,200  23,400  \n12,00,000  -85,800  -33,900  -23,400  18,200  \n13,00,000  -96,200  -49,400  -33,800  18,200  \n14,00,000  -1,06,600  -59,800  -44,200  18,200  \n15,00,000  -1,17,000  -70,200  -54,600  7,800  \n \nHow to opt for the new tax regime ? \nFrom the Assessment Year 2024 -25, the default tax regime will be the new tax regime  of'),
 Document(page_content='[As amended  by Finance  Act, 2023]   \n  \nNew  tax regime Rate for Individual , HUFs , AOP, BOI and AJP  \nNew tax regime (also known as alternative tax regime) is optional for the Assessment Year 2023 -24. \nAn individual or HUF has to exercise the option under section 115BAC(5) to avail its benefit.  \nHowever, fo r the Assessment Year 2024 -25, the new tax regime is the default tax regime for the \nIndividual or HUF. Further, the benefit of new tax regime has also extended

In [ ]:
llm = GooglePalm(temperature=0.1)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
prompt_template  = """
Use the following piece of context to answer the question. Please provide a detailed response for each of the question.

{context}

Question: {question}

Answer in English"""

In [ ]:
prompt = PromptTemplate(template = prompt_template , input_variables=["context", "question"])

In [ ]:
query = "what is the difference between old and new tax regimes"

In [ ]:
qa.run(query)

"The main difference between the old and new tax regimes is that the new tax regime does not allow for any exemptions or deductions, while the old tax regime does. This means that taxpayers who are eligible for certain exemptions or deductions under the old tax regime may be better off opting for that regime, while taxpayers who are not eligible for any exemptions or deductions may be better off opting for the new tax regime.\n\nIn addition, the new tax regime has a lower tax rate than the old tax regime, but the new tax regime also has a higher standard deduction. This means that taxpayers who have a lot of deductions may be better off opting for the old tax regime, while taxpayers who do not have a lot of deductions may be better off opting for the new tax regime.\n\nUltimately, the decision of which tax regime to opt for will depend on the individual taxpayer's circumstances. Taxpayers should carefully consider their options before making a decision."

In [ ]:
query = "what are the different tax slabs in new tax regime"

In [ ]:
qa.run(query)

'Upto 3,00,000  Nil\nFrom 3,00,001 to 6,00,000  5%\nFrom 6,00,001 to 9,00,000  10%\nFrom 9,00,001 to 12,00,000  15%'

In [ ]:
query = "what is the standard deduction in both new and old tax regimes"

In [ ]:
qa.run(query)

'The standard deduction in the old tax regime is Rs. 50,000 and in the new tax regime is Rs. 40,000.'

In [ ]:
query = "what are the exemptions allowed in old tax regimes"

In [ ]:
qa.run(query)

'The following are the exemptions allowed in old tax regimes:\n\n- Standard Deduction [Section 16(ia)]\n- Leave Travel concession [Section 10(5)]\n- House Rent Allowance [Section 10(13A)]\n- Official and personal allowances (other than those as \nmay be prescribed) [Section 10(14)]\n- Allowances to MPs/MLAs [Section 10(17)]\n- Entertainment Allowance [Section 16((ii)]\n- perquisites provided under any other law for the time \nbeing in force'

In [ ]:
query = "assume a person earns 12lpa. could you tell which tax plan will be beneficial: old or new tax regime"

In [ ]:
qa.run(query)

'New tax regime'

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: is HRA applicable in both regimes
Answer: Yes, HRA is applicable in both regimes
Input Prompt: what are the different tax slabs in old tax regime
Answer: The tax slabs in the old tax regime are as follows:

Net income range  Resident Super 
Senior Citizen  Resident Senior 
Citizen  Any other 
Individual

[As amended  by Finance  Act, 2023]
Input Prompt: lets assume a person earns 35lpa. how to calculate tax and find which tax regime will be beneficial
Answer: The tax on 35 lpa income under the new tax regime is 12,000 + 30% of (35,00,000 - 2,50,000) = 12,000 + 9,75,000 = 9,87,000.

The tax on 35 lpa income under the old tax regime is 3,000 + 20% of (35,00,000 - 2,50,000) = 3,000 + 6,00,000 = 6,00,300.

So, the old tax regime is more beneficial for a person earning 35 lpa.
Input Prompt: could you explain how you calculated the taxes
Answer: The taxes are calculated as follows:

1. The total income is first bifurcated into normal income and special income.
2. The normal inc

KeyboardInterrupt: ignored